### Import Libraries that will be needed

In [1]:
import pandas as pd
import datetime

### Import data from .csv and reformat time

In [2]:
df_raw = pd.read_csv("20170606_Combo2.csv")

#Convert Time column to datetime.time format
df_raw.Time = pd.to_datetime(df_raw.Time, format='%H:%M:%S').dt.time 

### Trim to 24 hour range

In [3]:
def get_data_range(time_list):
    """
    Find the start and end indices for a 24 hour range starting between 
    10:00am and 10:05am.
    Takes time_list - a list of datetime.time objects
    Returns a tuple: (start_index, end_index)
    *Selects first encountered instance of time between 
    10:00am and 10:05am as start index*
    """
    lower_limit = datetime.time(10,0,0)
    upper_limit = datetime.time(10,5,0)
    start_index = 0
    
    for i in range(len(time_list)):
        if time_list[i] >= lower_limit and time_list[i] < upper_limit:
            start_index = i
            break
            
    #288 is the number of rows until 24 hours later
    return (start_index, start_index + 288) 

In [4]:
# Trim raw data to 24 hour window
trim_start, trim_end = get_data_range(list(df_raw.Time))
df = df_raw[trim_start:trim_end]
df = df.reset_index(drop=True)
df_trimmed_copy = df.copy() # to be used in final output to excel 

### Create reference columns

In [5]:
# Create Boolean column indicating Night=True Day=False
# In most cases this should be just split into top/bottom half of df;
# implented as a function in case assumption is not met.
def is_night(x):
    """
    Take datetime.time object and return True if between 10pm and 10am;
    return False otherwise.
    """
    night_start = datetime.time(22,0,0)
    night_end = datetime.time(10,0,0)
    if (x >= night_start and x < datetime.time(23, 59, 59)) or x < night_end:
        return True
    else:
        return False

In [6]:
df.isNight = df.Time.apply(is_night)

In [7]:
# Create boolean columns for fly sleeping state
def create_sleeping_columns(df):
    """
    Creates a new boolean column for each fly. True indicates fly is asleep 
    (activity = 0) for that time point.
    Columns used for filters in data analysis. 
    """
    act_col = 11
    
    for i in range(5):
        df.loc[:,'fly' + str(i+1) + 'sleeping'] = df.iloc[:,i+act_col].apply(lambda x: x == 0)

In [8]:
create_sleeping_columns(df)

### Get day and night bouts and create dataframes

In [9]:
# Following code creates two new dataframes: all_day_bouts_df, all_night_bouts_df
# These will be exported to two different sheets in excel. 

def get_bout_indices(activity_list):
    """
    Takes a list, activity_list, and returns a list of tuples of 
    the start/end indices in which sleeping bouts occurr.
    I.e. if two sleeping bouts occured, the first from index 5 to 20,
    and the second from index 30 to 40, this function will return 
    [(5,20), (30,40)]
    """
    indices = []
    start_index = 1
    end_index = 1
    in_bout = False
    
    for i in range(len(activity_list)):
        if activity_list[i] == 0 and in_bout == False: 
            start_index = i
            in_bout = True
        if (activity_list[i] != 0 or i == len(activity_list)-1) and in_bout == True:
            end_index = i
            in_bout = False
            if i == len(activity_list)-1:
                indices.append((start_index, end_index+1))
            else:
                indices.append((start_index, end_index))
                
    return indices 

In [13]:
colnames = df.columns[11:16] # Ordered list of fly genotype name strings

def get_day_night_bouts(df, resp_colnum, activity_colnum):
    """
    Gets lists of mr rates for a single fly during during indepedent sleeping 
    bouts during either day or night. 
    Takes dataframe, column number for resp data, column number for sleep data.
    Returns a tuple of two lists of lists: resp data for daytime sleep bouts, 
    and resp data for nighttime sleep bouts
    """
    bout_indices = get_bout_indices(list(df.iloc[:,activity_colnum]))
    resp_bouts_day = []
    resp_bouts_night = []
    
    for i in bout_indices:
        if df.isNight[i[0]:i[1]].mean() >= 0.5:
            resp_bouts_night.append(list(df.iloc[:,resp_colnum])[i[0]:i[1]])
        else:
            resp_bouts_day.append(list(df.iloc[:,resp_colnum])[i[0]:i[1]])
            
    if not resp_bouts_day:
        resp_bouts_day = [['none']]
        
    if not resp_bouts_night:
        resp_bouts_night = [['none']]
        
    return resp_bouts_day, resp_bouts_night

def get_all_bouts(df, colnames):
    """
    Returns a tuple of two dictionaries containing individual bouts for each fly 
    for 1) day sleep, 2) night sleep
    Dictionary format - key: genotype; value: list of lists containing mr data
    for individual sleeping bouts
    """
    mr_col = 4
    act_col = 11
    static_act_col = 11
    colnames = df.columns[11:16]
    all_day_bouts = {x: [] for x in colnames}
    all_night_bouts = {x: [] for x in colnames}
    
    for i in range(5):
        day_bouts, night_bouts = get_day_night_bouts(df, mr_col, act_col)
        all_day_bouts[df.columns[i+static_act_col]] += day_bouts 
        all_night_bouts[df.columns[i+static_act_col]] += night_bouts 
        mr_col+=1
        act_col+=1
        
    return all_day_bouts, all_night_bouts

In [14]:
day_bouts_dict, night_bouts_dict = get_all_bouts(df, colnames)

In [15]:
day_bouts_dict

{'569': [[1.374187],
  [1.637229, 1.7406259999999998, 1.8160339999999999, 1.85493],
  [1.8894709999999999],
  [1.7615529999999999,
   1.7307240000000002,
   1.8120499999999999,
   1.783196,
   1.58762,
   1.7781310000000001,
   1.6899599999999999,
   1.6926259999999997,
   1.691657,
   1.6241270000000001,
   1.7165119999999998,
   1.7819830000000001,
   1.7278340000000001],
  [1.4348100000000001, 1.3689049999999998],
  [1.655532],
  [1.672607, 1.536208, 1.513528, 1.538921],
  [1.5822270000000001,
   1.5618920000000001,
   1.5083530000000001,
   1.3830579999999999,
   1.4364950000000001,
   1.5388280000000001],
  [1.423548,
   1.3003549999999999,
   1.234561,
   1.2104489999999999,
   1.1224100000000001],
  [1.3833870000000001],
  [1.472397, 1.3661290000000001, 1.291677],
  [1.4349459999999998,
   1.32497,
   1.3644450000000001,
   1.3066070000000001,
   1.3006309999999999,
   1.38141],
  [1.175584, 1.293239, 1.263099, 1.277228, 1.197057, 1.267401],
  [1.7131580000000002, 1.660366],
  [

In [16]:
def get_all_bouts_df(day_bouts_dict, night_bouts_dict, colnames):
    """
    Takes dictionaries day_bouts_dict and night_bouts_dict and returns 
    a tuple of two dataframes containing the information in the dictionaries 
    properly formatted for excel output.
    Different flies are seperated by an empty column. 
    """
    all_bouts_day_list = []
    all_bouts_night_list = []
    
    for i in colnames:
        # [[]] used to create empty lists between columns to indicate different flies
        all_bouts_day_list += day_bouts_dict[i] + [[]] 
        all_bouts_night_list += night_bouts_dict[i] + [[]]
        
    all_day_df = pd.DataFrame(all_bouts_day_list).transpose()
    all_night_df = pd.DataFrame(all_bouts_night_list).transpose()
    
    return all_day_df, all_night_df

In [17]:
# Create dataframes for day/night bouts
all_day_bouts_df, all_night_bouts_df = get_all_bouts_df(day_bouts_dict, night_bouts_dict, colnames)

In [18]:
all_day_bouts_df

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,1.38562,1.34319,1.47893,2.27211,None,1.97301,1.87442,None,1.36457,1.39231,...,1.4724,1.43495,1.17558,1.71316,1.67649,1.92164,1.63138,None,none,None
1,1.35791,1.40642,NaN,NaN,NaN,NaN,NaN,NaN,1.32977,1.32916,...,1.36613,1.32497,1.29324,1.66037,1.66671,1.73234,NaN,NaN,NaN,NaN
2,1.46432,1.57771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.43321,...,1.29168,1.36445,1.2631,NaN,1.95774,1.78534,NaN,NaN,NaN,NaN
3,1.42676,1.47957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.33056,...,NaN,1.30661,1.27723,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.40119,1.37687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.35054,...,NaN,1.30063,1.19706,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.50518,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.40591,...,NaN,1.38141,1.2674,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.6015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.56052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.41545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1.49845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Get sleep profiles and create dataframe

In [19]:
# Following code is used to create the sleep_profile dataframe, 
# which will be exported as it's own sheet in the excel output. 
def get_single_sleep_profile(df, resp_colnum, activity_colnum):
    """
    Returns a list of tuples with metabolic rate sums, the percentage sleep, 
    and total beam breaks per hour for 24 hours, for one individual fly.
    """
    fly_activity = df.iloc[:,activity_colnum]
    fly_resp = df.iloc[:,resp_colnum]
    hourly_resp_sleep = []
    
    for i in range(24):
        #12 rows are selected at a time to bin into 1-hour 
        activity = fly_activity[i*12:i*12+12] 
        resp = fly_resp[i*12:i*12+12]
        num_sleep_blocks = 0
        for i in activity:
            if i == 0:
                num_sleep_blocks += 1
        sleep_avg = num_sleep_blocks / 12 * 100
        hourly_resp_sleep.append((resp.sum(), sleep_avg, activity.sum()))
        
    return hourly_resp_sleep

def make_sleep_profile_colnames(df):
    """
    Create list of ordered column names for dataframe to be created 
    from sleep_profile dictionary
    """
    colnames = ['Time']
    
    for i in range(11,16):
        colnames.append(df.columns[i] + ' MR Sum')
        colnames.append(df.columns[i] + ' Avg Sleep')
        colnames.append(df.columns[i] + ' Beam Breaks')
        
    return colnames

def make_sleep_profile_df(df):
    """
    Returns a dictionary containing a time index and the sleep profiles 
    (both metabolic rate sum and average sleep) for each fly in the dataframe.
    """
    mr_col = 4
    act_col = 11
    sleep_profile_dict = {}
    sleep_profile_dict['Time'] = [df.Time[x*12-1] for x in range(1, 25)]
    
    def add_to_dict(name, index):
        sleep_profile_dict[df.columns[act_col] + name] = [x[index] for x in sleep_profile]
        
    for i in range(5):
        sleep_profile = get_single_sleep_profile(df, mr_col, act_col)
        add_to_dict(' MR Sum', 0)
        add_to_dict(' Avg Sleep', 1)
        add_to_dict(' Beam Breaks', 2)
        mr_col+=1
        act_col+=1
        
    sleep_profile_all = pd.DataFrame(sleep_profile_dict, 
                                     columns=make_sleep_profile_colnames(df))
    
    return sleep_profile_all

In [20]:
sleep_profile_df = make_sleep_profile_df(df)

In [21]:
sleep_profile_df

,Time,6326 MR Sum,6326 Avg Sleep,6326 Beam Breaks,6326.1 MR Sum,6326.1 Avg Sleep,6326.1 Beam Breaks,6326.2 MR Sum,6326.2 Avg Sleep,6326.2 Beam Breaks,569 MR Sum,569 Avg Sleep,569 Beam Breaks,569.1 MR Sum,569.1 Avg Sleep,569.1 Beam Breaks
0,10:59:02,25.675990,0.000000,525,21.162839,0.000000,555,15.331181,16.666667,404,20.526852,50.000000,63.0,17.845057,100.0,0.0
1,11:59:02,25.665004,0.000000,328,22.390615,16.666667,416,18.063755,0.000000,545,20.904515,58.333333,96.0,18.641777,100.0,0.0
2,12:59:02,24.392391,0.000000,418,20.535549,0.000000,473,18.930661,0.000000,333,19.331973,66.666667,44.0,17.611385,100.0,0.0
3,13:59:02,21.897831,0.000000,490,16.772490,0.000000,527,17.149652,0.000000,466,18.894350,66.666667,52.0,18.390482,100.0,0.0
4,14:59:02,20.113219,0.000000,479,15.057204,0.000000,466,16.755973,33.333333,255,16.412927,66.666667,82.0,18.244200,100.0,0.0
5,15:59:02,17.335780,0.000000,424,14.111465,0.000000,495,14.952431,16.666667,488,15.743438,33.333333,151.0,16.964227,100.0,0.0
6,16:59:03,17.673665,50.000000,215,14.076743,0.000000,486,14.705786,16.666667,223,14.880621,0.000000,278.0,15.373038,100.0,0.0
7,17:59:03,17.494353,75.000000,51,15.305499,0.000000,377,14.938866,0.000000,651,15.864150,50.000000,57.0,15.440606,100.0,0.0
8,18:59:03,17.377517,25.000000,187,15.477812,0.000000,352,14.678581,0.000000,603,15.031081,50.000000,139.0,16.126881,100.0,0.0
9,19:59:03,16.994248,16.666667,304,14.354978,0.000000,410,12.506348,0.000000,514,13.507639,16.666667,211.0,13.304900,100.0,0.0


### Create summary dataframes

In [22]:
# Following code is used to analyze the data processed above, and generate 
# three dataframes, which will become three sheets in the final excel output. 

#Total Sleep 
def get_sleep_minutes_df(df, colnames):
    """
    Create summary df of minutes of sleep for each fly. 
    Exported as sheet to excel.
    """
    total_sleep_list = [[x, ] for x in colnames]
    sleep_col = 16
    column_names = ['Fly', 
                    'Total Sleep Min',
                    'Total Day Sleep Min', 
                    'Total Night Sleep Min']
    
    def add_to_list(data):
        total_sleep_list[i].append(data)
    
    for i in range(len(colnames)):
        sleep_ser = df.iloc[:,i+sleep_col]
        # data are in 5min bins, multiply by 5 to make per minute
        add_to_list(sleep_ser.sum()*5) 
        add_to_list(sleep_ser[df.isNight == False].sum()*5)
        add_to_list(sleep_ser[df.isNight == True].sum()*5)
        
    return pd.DataFrame(total_sleep_list, columns=column_names)

In [23]:
sleep_minutes_df = get_sleep_minutes_df(df, colnames)

In [24]:
sleep_minutes_df

,Fly,Total Sleep Min,Total Day Sleep Min,Total Night Sleep Min
0,6326,380,100,280
1,6326.1,660,10,650
2,6326.2,655,100,555
3,569,1005,310,695
4,569.1,1440,720,720


In [25]:
# Wake Sleep MR
def get_wake_sleep_mr_df(df, colnames):
    """
    Create summary df of average metabolic rates during selected 
    time periods for each fly. Exported as sheet to excel. 
    """
    wake_sleep_mr_list = [[x, ] for x in colnames]
    mr_col = 4
    sleep_col = 16
    column_names = ['Fly',
                    'Mean Wake MR',
                    'Mean Sleep MR',
                    'Mean Day MR', 
                    'Mean Night MR', 
                    'Mean Day Sleep MR',
                    'Mean Night Sleep MR']
    
    def add_to_list(data):
        wake_sleep_mr_list[i].append(data)
    
    for i in range(len(colnames)):
        mr_ser = df.iloc[:,i+mr_col]
        sleep_ser = df.iloc[:,i+sleep_col]
        
        add_to_list(mr_ser[sleep_ser == False].mean())
        add_to_list(mr_ser[sleep_ser == True].mean())
        add_to_list(mr_ser[df.isNight == False].mean())
        add_to_list(mr_ser[df.isNight == True].mean())
        add_to_list(mr_ser[sleep_ser == True][df.isNight == False].mean())
        add_to_list(mr_ser[sleep_ser == True][df.isNight == True].mean())            
        
    return pd.DataFrame(wake_sleep_mr_list, columns=column_names)

In [26]:
wake_sleep_mr_df = get_wake_sleep_mr_df(df, colnames)

In [27]:
wake_sleep_mr_df

,Fly,Mean Wake MR,Mean Sleep MR,Mean Day MR,Mean Night MR,Mean Day Sleep MR,Mean Night Sleep MR
0,6326,1.702412,1.294715,1.817698,1.373554,1.496256,1.222735
1,6326.1,1.489342,1.189635,1.489933,1.214028,1.923714,1.178341
2,6326.2,1.293183,0.976536,1.377711,0.921768,1.424398,0.895840
3,569,1.461414,1.158807,1.507883,0.993202,1.541266,0.988214
4,569.1,NaN,1.342311,1.483156,1.203422,1.483156,1.203422


In [28]:
def get_mean_hourly_sleep_mr_df(sleep_profile, colnames):
    """
    Creates summary df for mean hourly sleeping metabolic rate for each fly. 
    Exported as sheet to excel. 
    """
    mr_hourly = sleep_profile.iloc[:,1::3]
    # sleep profile is split into top half: day; bottom half: night
    night_bool_list = [x > len(mr_hourly)/2-1 for x in range(len(mr_hourly))]
    mr_hourly.isNight = pd.Series(night_bool_list)      
    mean_hourly_list = [[x, ] for x in colnames]
    column_names = ['Fly', 
                    'Mean Hourly MR Total', 
                    'Mean Hourly MR Day', 
                    'Mean Hourly MR Night']
    
    def add_to_list(data):
        mean_hourly_list[i].append(data)
    
    for i in range(len(colnames)):
        mr_ser = mr_hourly.iloc[:,i]
        add_to_list(mr_ser.mean())
        add_to_list(mr_ser[mr_hourly.isNight == False].mean())
        add_to_list(mr_ser[mr_hourly.isNight == True].mean())
        
    return pd.DataFrame(mean_hourly_list, columns=column_names)

In [29]:
mean_hourly_sleep_mr_df = get_mean_hourly_sleep_mr_df(sleep_profile_df, colnames)

In [30]:
mean_hourly_sleep_mr_df

,Fly,Mean Hourly MR Total,Mean Hourly MR Day,Mean Hourly MR Night
0,6326,18.996038,21.509424,16.482652
1,6326.1,16.099603,17.630873,14.568332
2,6326.2,13.682065,16.302918,11.061213
3,569,14.880851,17.843279,11.918423
4,569.1,15.995869,17.550680,14.441058


## Export to .xlsx

In [32]:
# Convert datetime.time columns back to str for correct formatting in Excel
sleep_profile_df.Time = sleep_profile_df.Time.astype(str)
df_trimmed_copy.Time = df_trimmed_copy.Time.astype(str)

def excel_out(df, name, **kwargs):
    """
    Wrapper function - Takes a dataframe, and a desired sheet name (str).
    Sends to new sheet in excel output
    """
    df.to_excel(writer, sheet_name = name, index = False, **kwargs)
    
writer = pd.ExcelWriter('sample_out.xlsx', engine='xlsxwriter')
excel_out(df_trimmed_copy, 'Trimmed Analysis')
excel_out(all_day_bouts_df, 'All Day Bouts', header = False)
excel_out(all_night_bouts_df, 'All Night Bouts', header = False)
excel_out(sleep_profile_df, 'Sleep Profile')
excel_out(sleep_minutes_df, 'Min. of Sleep')
excel_out(wake_sleep_mr_df, 'Wake Sleep MR')
excel_out(mean_hourly_sleep_mr_df, 'Mean Hourly Sleep')
writer.save()